In [3]:
import torch
import numpy as np
from smplx import SMPLX
from smotdm.renderer.matplotlib import MatplotlibRender
from smotdm.utils import loop_interx
from smotdm.rifke import joints_to_rifke, rifke_to_joints


INTERX_DATASET_ROOT = "deps/interx"
device = torch.device("cpu")

renderer = MatplotlibRender(jointstype="smplxjoints")

In [4]:
iterator = loop_interx(INTERX_DATASET_ROOT)


for scene_id, num_frames, motions, texts in iterator:
    smplx_model = SMPLX(
        model_path="deps/smplx/SMPLX_NEUTRAL.npz",
        batch_size=num_frames,
        num_betas=10,
        use_pca=False,
        use_face_contour=True,
    )
    # joints[..., 0] *= -1
    # joints_m = swap_left_right(joints)

    feats = []
    for motion in motions:
        output = smplx_model(
            **motion,
            return_full_pose=True,
        )
        # XZY -> XYZ
        joints = torch.from_numpy(
            np.dot(
                output.joints.detach().cpu().numpy(),
                np.array([[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]]),
            )
        )

        feats.append(joints_to_rifke(joints[:, : smplx_model.NUM_JOINTS, :]))
    scene_motions = torch.stack(feats)
    break

scene_motions.shape

  0%|          | 0/11388 [00:00<?, ?it/s]

torch.Size([2, 548, 163])

In [5]:
texts

["Two people stand face to face. One person extends his/her left hand to shake the other person's right hand, and then he/she extends his/her right hand to hold both of the other person's hands.",
 'The two people stand face to face and shake hands with their right hand at first, then proceed to use both hands for a warmer handshake.',
 'One person approaches the other person and they exchange a firm handshake, using both hands.']

In [6]:
import h5py

d = h5py.File("t.h5", "w")
d.create_dataset("123", data=scene_motions[:, :num_frames, :].detach().cpu().numpy())
d.close()

In [7]:
renderer(
    rifke_to_joints(scene_motions[0]).detach().cpu().numpy(),
    rifke_to_joints(scene_motions[1]).detach().cpu().numpy(),
    output="test.mp4",
)